In [6]:
# 한국어 문장 처리 모델
# 챗봇의 유사도 계산
# https://github.com/kairess/mental-health-chatbot/blob/master/simple_chatbot.ipynb
import pandas as pd
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity

In [7]:
# SentenceBERT 모델 로드
# 구글에서 개발한 NLP(자연어처리) 훈련모델, 자연어 처리 분야에서 좋은 성능 Language Model
# SentenceTransformer에서 여러 모델중 1개를 사용
model = SentenceTransformer('jhgan/ko-sroberta-multitask')

sentences = ["안녕하세요?", "한국어 문장 임베딩을 위한 버트 모델입니다."]
# 글자를 숫자벡터화 : 안녕하세요 숫자로 변함
embeddings = model.encode(sentences)

print(embeddings)

[[-0.37510458 -0.7733839   0.5927711  ...  0.57923526  0.3268346
  -0.65089655]
 [-0.09361681 -0.18191518 -0.19230832 ... -0.03165777  0.30412534
  -0.2679363 ]]


In [8]:
# 데이터 불러오기
df = pd.read_csv('ChatBotData.csv')
df.head()

,Q,A,label
0,12시 땡!,하루가 또 가네요.,0
1,1지망 학교 떨어졌어,위로해 드립니다.,0
2,3박4일 놀러가고 싶다,여행은 언제나 좋죠.,0
3,3박4일 정도 놀러가고 싶다,여행은 언제나 좋죠.,0
4,PPL 심하네,눈살이 찌푸려지죠.,0


In [9]:
# 컬럼 Unnamed: 3 삭제
df = df.drop(columns=['label'])
df.head()

,Q,A
0,12시 땡!,하루가 또 가네요.
1,1지망 학교 떨어졌어,위로해 드립니다.
2,3박4일 놀러가고 싶다,여행은 언제나 좋죠.
3,3박4일 정도 놀러가고 싶다,여행은 언제나 좋죠.
4,PPL 심하네,눈살이 찌푸려지죠.


In [10]:
# isna() : 데이터 프레임 결측 값을 확인, 그 결과를 True, False로 반환
# 3,4의 NaN 데이터가 모두 삭제됨.
df = df[~df['A'].isna()]
# df.to_csv('wellness_dataset_ok.csv') # 파일저장
df.head()

,Q,A
0,12시 땡!,하루가 또 가네요.
1,1지망 학교 떨어졌어,위로해 드립니다.
2,3박4일 놀러가고 싶다,여행은 언제나 좋죠.
3,3박4일 정도 놀러가고 싶다,여행은 언제나 좋죠.
4,PPL 심하네,눈살이 찌푸려지죠.


In [11]:
# 0번째 row의 유저 컬럼 내용 출력
df.loc[0, 'Q']

'12시 땡!'

In [12]:
model.encode(df.loc[0, 'Q'])[:5]

array([-0.19072711,  0.32147592,  0.59873784,  0.4461787 , -0.25963923],
      dtype=float32)

In [13]:
len(df)

11823

In [14]:

# 1034개 1차원 배열 생성
df['embedding'] = pd.Series([[]] * len(df)) # dummy
# 모델인코딩 후, 유저컬럼 글자들이 embedding컬럼에 벡터화, list저장
df['embedding'] = df['Q'].map(lambda x: list(model.encode(x)))
# df.to_csv('wellness_dataset_ok.csv') # embedding컬럼 추가해서 파일저장

df.head()

,Q,A,embedding
0,12시 땡!,하루가 또 가네요.,"[-0.19072711, 0.32147592, 0.59873784, 0.446178..."
1,1지망 학교 떨어졌어,위로해 드립니다.,"[0.123206064, 0.64236826, 0.68536025, -0.42912..."
2,3박4일 놀러가고 싶다,여행은 언제나 좋죠.,"[0.024357582, -0.22393535, 0.22917657, -0.4501..."
3,3박4일 정도 놀러가고 싶다,여행은 언제나 좋죠.,"[0.03628536, -0.21048243, 0.07269376, -0.54036..."
4,PPL 심하네,눈살이 찌푸려지죠.,"[0.02203077, 0.70552164, -0.17157197, 0.165971..."


In [15]:
# 테스트 파일
text = '기분이 우울해'
# 모델링
embedding = model.encode(text)

In [16]:
answer = df.loc[3]
answer

Q                                              3박4일 정도 놀러가고 싶다
A                                                  여행은 언제나 좋죠.
embedding    [0.03628536, -0.21048243, 0.07269376, -0.54036...
Name: 3, dtype: object

In [17]:
# 개별적인 질문 함수호출
def return_answer(question):
    # 입력된 질문 벡터화
    embedding2 = model.encode(question)
    print("타입 : ",type(embedding2))
    # 모든 질문과 입력된 질문의 유사도 저장
    df['similarity'] = df['embedding'].map(lambda x: cosine_similarity([embedding2], [x]).squeeze())
    # 가장 높은 유사도 글을 리턴
    answer = df.loc[df['similarity'].idxmax()]
    return answer

In [18]:
# 테스트 파일
text2 = '완전 기분좋은 하루에요'
answer = return_answer(text2)
print("answer : ",answer)

print('유사한 질문 : ', answer['Q'])
print('챗봇 답변 : ', answer['A'])
print('유사도 : ', answer['similarity'])

타입 :  <class 'numpy.ndarray'>
answer :  Q                                                      오늘 기분 좋아
A                                                       저도 좋아요.
embedding     [-0.04852491, -0.34683305, 0.6035186, 0.468637...
similarity                                             0.836249
Name: 3273, dtype: object
유사한 질문 :  오늘 기분 좋아
챗봇 답변 :  저도 좋아요.
유사도 :  0.8362489342689514
